Using the available csv files and by scraping data from the website besoccer.com, I have compiled a dataset of 161,112 football matches in 14 leagues with 55 columns of relevant information about them.

In [1]:
import pandas as pd
data = pd.read_csv('cleaned_dataset.csv')
data.columns

Index(['Result', 'Season', 'Round', 'Teams_in_League',
       'Home_Team_Goals_For_This_Far', 'Home_Team_Goals_Against_This_Far',
       'Away_Team_Goals_For_This_Far', 'Away_Team_Goals_Against_This_Far',
       'Home_Team_Points', 'Away_Team_Points', 'Home_Team_Losing_Streak',
       'Away_Team_Losing_Streak', 'Home_Team_Winning_Streak',
       'Away_Team_Winning_Streak', 'Home_Team_Unbeaten_Streak',
       'Away_Team_Unbeaten_Streak', 'Elo_home', 'Elo_away',
       'Home_Wins_This_Far', 'Home_Draws_This_Far', 'Home_Losses_This_Far',
       'Away_Wins_This_Far', 'Away_Draws_This_Far', 'Away_Losses_This_Far',
       'Home_Wins_This_Far_at_Home', 'Home_Draws_This_Far_at_Home',
       'Home_Losses_This_Far_at_Home', 'Home_Wins_This_Far_Away',
       'Home_Draws_This_Far_Away', 'Home_Losses_This_Far_Away',
       'Away_Wins_This_Far_at_Home', 'Away_Draws_This_Far_at_Home',
       'Away_Losses_This_Far_at_Home', 'Away_Wins_This_Far_Away',
       'Away_Draws_This_Far_Away', 'Away_Losses_Thi

These contain information useful to a machine learning model for predicting future results.